## 모델 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers;
!pip install sentencepiece;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.0 MB/s eta 0:00:00


In [ ]:
from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)
import pickle
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.tokenization_kobert import KoBertTokenizer
#from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.lib import tokenization_kobert
import json
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [ ]:
%cd /content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai
# with open('/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/save_src_tk.pickle','rb') as f:
#     src_tokenizer = pickle.load(f)
# there's an error in this code when activating in local

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai


In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/kobert_tokenizer_add/added_tokens.json", 'rb') as f:
    dw_dict = json.load(f)
dict_swap = {v:k for k,v in dw_dict.items()}
ma=12290
mi=8002
for i in range(8002,12291):
    src_tokenizer.add_tokens(dict_swap[i])

In [ ]:
model = EncoderDecoderModel.from_pretrained('leadawon/ossp-v0_3')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

## 테스트데이터 불러오기

In [ ]:
def read_input(path):
    with open(path, 'rb') as f:
        test_dict = json.load(f)
    test_standard = []
    test_dialect = []
    for pairs in test_dict['utterance']:
        test_standard.append(pairs['standard_form'])
        test_dialect.append(pairs['dialect_form'])

    return test_standard ,test_dialect

In [ ]:
test_s ,test_d= read_input("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/test_data_big.json")

In [ ]:
def translate(text):
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    output = model.generate(**embeddings)[0, 1:-1]
    tmp = trg_tokenizer.decode(output.cpu())
    return tmp

In [ ]:
print("size of full : ",len(test_s))

size of full :  12451


In [ ]:
for i in range(len(test_s)):
    sr_text = test_d[i] # dialect
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    print("result :      ",tg_text)
    print("ground_truth :",test_s[i])
    print()
    print()
    if i%50==0:
        print("iteration num : ",i)
        break

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


source_text :  언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터
result :       언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터     
ground_truth : 언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터


iteration num :  0


## scoring

In [ ]:
!pip install "sacrebleu[ko]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mecab-ko-dic: filename=mecab_ko_dic-1.0.0-py3-none-any.whl size=33424394 sha256=19426d9f2ebe97480c10894ec1d12dcc3a36a1f5754a9748fde4f5c12b0fc4df
  Stored in directory: /root/.cache/pip/wheels/0d/46/b8/996432c9998690a5a66fb82f6499222e48a23ff5b6c7d80322
Successfully built mecab-ko-dic


In [ ]:
from sacrebleu.metrics import BLEU

In [ ]:
def score_B(sr_text, tg_text):

    candidate=[tg_text]
    reference=[[sr_text]]

    bleu = BLEU()

    bleuscore = bleu.corpus_score(candidate,reference).score
    return bleuscore
    #bleu_save.append(bleuscore)

In [ ]:
for i in range(len(test_s)):
    sr_text = test_d[i] # dialect
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    print("result      : ",tg_text)
    print("ground_truth: ",test_s[i])
    print("bleu_socre  :  {:.2f}".format(score_B(tg_text.strip(),test_s[i].strip())))
    print()
    print()
    if i%50==0:
        print("iteration num : ",i)

## chatgpt api test

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00


In [32]:
import openai
import json
import time

In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/chatgptapikey.json", 'rb') as f:
    tempjson = json.load(f)
    YOUR_API_KEY = tempjson["api_key"]

In [44]:
def ChatGPT(prompt, API_KEY=YOUR_API_KEY):
    # api key 세팅
    openai.api_key = API_KEY

    # ChatGPT API 호출 및 최신 언어 모델인 text-davinci-003을 가져옴
    completion = openai.Completion.create(
        engine='text-davinci-003'  # 'text-curie-001'  # 'text-babbage-001' #'text-ada-001' # 'gpt-3.5-turbo'
        , prompt=prompt
        , temperature=0.5
        , max_tokens=64
        , top_p=1
        , frequency_penalty=0
        , presence_penalty=0
)

    return completion['choices'][0]['text']

In [33]:
ChatGPT("이거 공짜로 해주면 안될까? ㅠㅠ")

'\n\n공짜로 해주는 것은 불가능합니다. 각 사람의 상황에 따라 다른 가격으로 제공하기 때문에 공짜로 해주는 것은 불가능합니다.'

In [40]:
out_json = {"tp":[("제주방언","방언chatgpt","번역모델","번역모델chatgpt","정답")]}

In [46]:
prompt_helper = "이 말을 이해했으면 1 아니면 0을 말해줘"
#prompt_helper = ""

In [47]:
for i in range(len(test_s)):
    sr_text = test_d[i] # dialect
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    di_gpt = ChatGPT(sr_text+prompt_helper)
    print("chatgpt res : ",di_gpt)
    print("result :      ",tg_text)
    st_gpt = ChatGPT(tg_text+prompt_helper)
    print("chatgpt res : ",st_gpt)
    print("ground_truth :",test_s[i])
    print()
    print()
    out_json["tp"].append((sr_text,di_gpt,tg_text,st_gpt,test_s[i]))
    if i%50==0:
        print("iteration num : ",i)
    if i==500:
        print("end")
        break


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

source_text :  일 년 모아야 되고  겅 하고 또
chatgpt res :  

이해하지 못했습니다. 문장을 좀 더 명확하게 작성해주세
result :       일 년 모아야 되고 그렇게 하고 또          
chatgpt res :  

0
ground_truth : 일 년 모아야 되고  그렇게 하고 또


source_text :  경 했잖아 이
chatgpt res :  

1
result :       그렇게 했잖아 이             
chatgpt res :  

1
ground_truth : 그렇게 했잖아 이


source_text :  할머니 할아버지가 봉투에 미리 다 담아그내
chatgpt res :  

할머니 할아버지가 봉투에 미리 다 담아그내이 말을 이
result :       할머니 할아버지가 봉투에 미리 다 담아서        
chatgpt res :  

0
ground_truth : 할머니 할아버지가 봉투에 미리 다 담아서


source_text :  아이덜 세배하면 봉투에 담앙
chatgpt res :  

0
result :       아이들 세배하면 봉투에 담아서         
chatgpt res :  

0
ground_truth : 아이들 세배하면 봉투에 담아서


iteration num :  50
source_text :  옷 사입은 거 이 입엉 가그내 세배해그내 세뱃돈 받고
chatgpt res :  

1
result :       옷 사입은 거 이 입고 가서 세배해서 세뱃돈 받고   
chatgpt res :  

0
ground_truth : 옷 사입은 거 이 입고 가서 세배해서 세뱃돈 받고


source_text :  아 근데 그게 세월이 많이 바 변해선게
chatgpt res :  

1
result :       아 근데 그게 세월이 많이 바 변했더라      
chatgpt res :  

1
ground

In [54]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/chatgptapitest_result.json", "w") as outfile:
    json.dump(out_json, outfile,ensure_ascii=False)

In [53]:
for i,v in enumerate(out_json["tp"]):
    out_json["tp"][i] = (v[0].strip(),v[1].strip(),v[2].strip(),v[3].strip(),v[4].strip())